In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import chain
from pathlib import Path

params = {
    'axes.grid' : True,
    "grid.linestyle": '--',
    "font.family": "serif",
    "font.serif": "Times New Roman",
}

sns.set_style("ticks", params)
sns.set_context("paper", font_scale=1.5)
sns.set_palette("Set2")

In [ ]:
pretrained_root_path = Path("<path to pretrained 7B flores results>")

In [ ]:
def load_scores(scores_file: Path):
    lines = scores_file.read_text().splitlines()
    scores = {}
    for line in lines:
        key, value = line.split(": ")
        scores[key] = float(value)
    return scores

def load_lp(data_root: Path, lp: str, instructions, ckpt: str):
    sys_scores_path = data_root / lp / ckpt / instructions / "sys_scores.txt"
    scores = load_scores(sys_scores_path)
    return {"lp": lp, "Direction": "En-XX" if lp.startswith("en") else "XX-En", **scores}


def load_results(data_root: Path, instructions, ckpt: str):
    results = []
    lps_dirs = [d for d in data_root.iterdir() if d.is_dir()]
    for lp_dir in lps_dirs:
        lp = lp_dir.name
        results.append(load_lp(data_root, lp, instructions, ckpt))
    return pd.DataFrame(results)

pd.options.display.float_format = '{:,.2f}'.format
pretrained_results = load_results(pretrained_root_path, "zero_shot_instructions", "0")
pretrained_results.drop(columns=["Direction"])
pretrained_results["COMET-22"] *= 100
pretrained_results["COMETKiwi"] *= 100
pretrained_results